In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Load CSV file into a DataFrame
df = pd.read_csv('data/titanic.csv')
df.head()

In [ ]:
df.drop('Name', axis=1, inplace=True)
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(10, 3))
sns.histplot(df['Survived'], kde=False)
plt.title('Survived')
plt.xlabel('Survived')
plt.ylabel('Count')
plt.xticks([0, 1], ['0', '1'])
plt.show()

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=4, figsize=(20, 20))
for i, ax in zip(df.columns, axes.flat):
    sns.histplot(x=df[i], ax=ax).set(title=f"Histogram of : {i}", xlabel="")
plt.show()

In [ ]:
df.info()

In [ ]:
# Assuming df is your DataFrame
df_without_sex = df.drop('Sex', axis=1)  # Drop the 'Sex' column

# Calculate the correlation matrix using Spearman method
corr = df_without_sex.corr(method='spearman')

# Display the correlation matrix
print(corr)


In [ ]:
# Define features (X) and target (y)
X = df.drop('Survived', axis=1)
y = df['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Display train and test data
df_train, df_test, y_train, y_test = train_test_split(df, df['Survived'], test_size=0.2, random_state=123)

print("\nTRAIN:")
display(df_train.head())
print("SHAPE:", df_train.shape)

print("\nTEST:")
display(df_test.head())
print("SHAPE:", df_test.shape)

In [ ]:
df_train.info()
df_train.isna().sum()
df_train.describe()

In [ ]:
plt.figure(figsize=(10,10))
sns.set(font_scale=1.25)
sns.heatmap(
    corr, linewidths=1.5, annot=True, square=True, fmt=".2f", annot_kws={"size": 10}
)
plt.show()

In [ ]:
df_train.head()

In [ ]:
y = df_train["Survived"].values
print(y.shape)

In [ ]:
y = df_train["Survived"].values

In [ ]:
print(y.shape)

In [ ]:
# Visualize the distribution of the target variable 'Survived'
sns.countplot(x='Survived', data=df)
plt.xlabel('Survived')
plt.ylabel('Count')
plt.title('Distribution of Survived')
plt.show()

# Explore relationships between features and the target variable
sns.countplot(x='Survived', hue='Pclass', data=df)
plt.xlabel('Survived')
plt.ylabel('Count')
plt.title('Survival by Passenger Class')
plt.show()

sns.boxplot(x='Survived', y='Age', data=df)
plt.xlabel('Survived')
plt.ylabel('Age')
plt.title('Survival by Age')
plt.show()

In [ ]:
from scipy.stats import chi2_contingency

# Chi-squared test for Sex vs. Survived
sex_survived_contingency = pd.crosstab(df['Sex'], df['Survived'])
chi2_stat, p_val, dof, expected = chi2_contingency(sex_survived_contingency)
print("Chi-squared p-value for Sex vs. Survived:", p_val)

# Chi-squared test for Pclass vs. Survived
pclass_survived_contingency = pd.crosstab(df['Pclass'], df['Survived'])
chi2_stat, p_val, dof, expected = chi2_contingency(pclass_survived_contingency)
print("Chi-squared p-value for Pclass vs. Survived:", p_val)


In [ ]:
from scipy.stats import f_oneway

# Conduct ANOVA for Age and Pclass
anova_results = f_oneway(
    df_train[df_train['Pclass'] == 1]['Survived'].dropna(),
    df_train[df_train['Pclass'] == 2]['Survived'].dropna(),
    df_train[df_train['Pclass'] == 3]['Survived'].dropna()
)

print("ANOVA p-value for Survival and Pclass:", anova_results.pvalue)


In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='Survived', y='Age', data=df_train)
plt.xlabel('Survived')
plt.ylabel('Age')
plt.title('Distribution of Age by Survival')
plt.show()


In [ ]:
df_train.describe()

In [ ]:
df_train.tail()

In [ ]:
import category_encoders as ce

# Resetting indices for X_train and y_train
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

# Example 1: Target Encoding
encoder_target = ce.TargetEncoder(cols=['Pclass', 'Sex'])
X_train_encoded_target = encoder_target.fit_transform(X_train, y_train)
X_test_encoded_target = encoder_target.transform(X_test)

# Example 2: CatBoost Encoding
encoder_catboost = ce.CatBoostEncoder(cols=['Pclass', 'Sex'])
X_train_encoded_catboost = encoder_catboost.fit_transform(X_train, y_train)
X_test_encoded_catboost = encoder_catboost.transform(X_test)

# Example 3: Binary Encoding
encoder_binary = ce.BinaryEncoder(cols=['Pclass', 'Sex'])
X_train_encoded_binary = encoder_binary.fit_transform(X_train, y_train)
X_test_encoded_binary = encoder_binary.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the model
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_encoded_binary, y_train)

# Predict using the model
y_pred = log_reg_model.predict(X_test_encoded_binary)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
# Resetting indices for X_train and y_train
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

# Initialize the encoder and apply target encoding
encoder_target = ce.TargetEncoder(cols=['Pclass', 'Sex'])
X_train_encoded = encoder_target.fit_transform(X_train, y_train)
X_test_encoded = encoder_target.transform(X_test)

# Initialize and train the model
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_encoded, y_train)

# Predict using the model
y_pred = log_reg_model.predict(X_test_encoded)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))
